# Run MPNN interface design on the bound states

### Imports

In [3]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
running in directory: /home/broerman/crispy_shifty/projects/OPS/round_1/design
running on node: sofia


### Run MPNN on the interfaces

In [2]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "05_mpnn_bound_states"
design_list_file = "/home/broerman/crispy_shifty/projects/OPS/round_1/design/04_loop_bound_states/looped_states.list"
output_path = os.path.join(
    "/home/broerman/crispy_shifty/projects/OPS/round_1/design", simulation_name
)
options = " ".join(
    [
        "out:level 200",
    ]
)
extra_kwargs = {"mpnn_design_area": "scan", "num_sequences": "10", "batch_size": "10"}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_bound_state",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="6G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Run the following command with your desired environment active:
sbatch -a 1-2423 /home/broerman/crispy_shifty/projects/OPS/round_1/design/05_mpnn_bound_states/run.sh


### Collect scorefiles of the MPNN designed bound states and concatenate

In [4]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "05_mpnn_bound_states"
output_path = os.path.join(
    "/home/broerman/crispy_shifty/projects/OPS/round_1/design", simulation_name
)

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile

In [5]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

simulation_name = "05_mpnn_bound_states"
output_path = os.path.join(
    "/home/broerman/crispy_shifty/projects/OPS/round_1/design", simulation_name
)

scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
scores_df = scores_df.convert_dtypes()

  0%|          | 0/7269 [00:00<?, ?it/s]

### Data exploration

In [7]:
print(len(scores_df))
print(list(scores_df.columns))

7269
['bb_clash', 'bb_clash_delta', 'buns_parent', 'cms_AcB', 'cms_AnAc', 'cms_AnAcB', 'cms_AnB', 'docked_helix', 'dssp', 'exposed_hydrophobics_parent', 'fixed_resis', 'geometry_parent', 'holes_all_parent', 'holes_core_parent', 'loop_dist', 'loop_sc', 'looped_length', 'mismatch_probability_parent', 'mpnn_design_area', 'mpnn_seq_0000', 'mpnn_seq_0001', 'mpnn_seq_0002', 'mpnn_seq_0003', 'mpnn_seq_0004', 'mpnn_seq_0005', 'mpnn_seq_0006', 'mpnn_seq_0007', 'mpnn_seq_0008', 'mpnn_seq_0009', 'mpnn_seq_0010', 'mpnn_temperature', 'new_loop_str', 'packstat_parent', 'parent', 'parent_length', 'path_in', 'pdb', 'pivot_helix', 'pre_break_helix', 'repeat_len', 'sap_parent', 'sc_AcB', 'sc_AnAc', 'sc_AnAcB', 'sc_AnB', 'sc_all_parent', 'score', 'score_per_res', 'score_per_res_parent', 'shift', 'ss_sc', 'state', 'time', 'topo', 'total_length', 'total_score', 'trim_n', 'trimmed_length']


### Save individual fastas

In [8]:
sys.path.insert(0, "/home/broerman/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

simulation_name = "05_mpnn_bound_states"
output_path = os.path.join(
    "/home/broerman/crispy_shifty/projects/OPS/round_1/design", simulation_name
)

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq")

  0%|          | 0/7269 [00:00<?, ?it/s]

### Save a list of outputs
Sort by length

In [9]:
simulation_name = "05_mpnn_bound_states"
output_path = os.path.join(
    "/home/broerman/crispy_shifty/projects/OPS/round_1/design", simulation_name
)

scores_df = scores_df.sort_values("looped_length")

with open(os.path.join(output_path, "mpnn_states.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

  0%|          | 0/7269 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [10]:
simulation_name = "05_mpnn_bound_states"
output_path = os.path.join(
    "/home/broerman/crispy_shifty/projects/OPS/round_1/design", simulation_name
)

with open(os.path.join(output_path, "mpnn_states.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/7269 [00:00<?, ?it/s]